In [ ]:
import pandas as pd
import numpy as np
from math import floor, sqrt

# -------------------------------
# 辅助函数
# -------------------------------
def compute_vwap(df):
    """计算当日逐分钟累计的VWAP"""
    df = df.copy()
    df['cum_vol'] = df['Volume'].cumsum()
    df['cum_pv'] = (df['Close'] * df['Volume']).cumsum()
    df['VWAP'] = df['cum_pv'] / df['cum_vol']
    return df

def simulate_day(day_df, allowed_times, position_size):
    """
    模拟单个交易日内的交易（基于当前边界与VWAP平仓逻辑）。
    
    参数：
      day_df: 当日所有分钟数据，要求已包含 DateTime, Time, Close, UpperBound, LowerBound, VWAP
      allowed_times: 允许触发交易的时间点列表
      position_size: 当日固定的交易股数（根据资金和VIX动态仓位计算得出）
    
    返回：
      trades: 当日的所有交易记录，包含 entry_time, exit_time, side, entry_price, exit_price, pnl（美元盈亏）
    """
    position = 0   # 0 表示空仓，1 表示多仓，-1 表示空仓
    entry_price = np.nan
    trailing_stop = np.nan
    trade_entry_time = None
    trades = []
    
    for idx, row in day_df.iterrows():
        current_time = row['Time']
        price = row['Close']
        upper = row['UpperBound']
        lower = row['LowerBound']
        vwap = row['VWAP']
        
        # 开仓信号仅在允许的时刻触发
        if position == 0 and current_time in allowed_times:
            if price > upper:
                position = 1
                entry_price = price
                trade_entry_time = row['DateTime']
                trailing_stop = max(upper, vwap)  # 多仓止损：取上边界和VWAP中较大值
            elif price < lower:
                position = -1
                entry_price = price
                trade_entry_time = row['DateTime']
                trailing_stop = min(lower, vwap)  # 空仓止损：取下边界和VWAP中较小值
        
        # 如果已有持仓，则更新VWAP和动态止损，并检查平仓信号
        if position != 0:
            if position == 1:
                new_stop = max(upper, vwap)
                trailing_stop = max(trailing_stop, new_stop)
                if price < trailing_stop:
                    exit_time = row['DateTime']
                    pnl = position_size * (price - entry_price)  # 多仓盈亏计算
                    trades.append({
                        'entry_time': trade_entry_time,
                        'exit_time': exit_time,
                        'side': 'Long',
                        'entry_price': entry_price,
                        'exit_price': price,
                        'pnl': pnl
                    })
                    position = 0
                    trailing_stop = np.nan
            elif position == -1:
                new_stop = min(lower, vwap)
                trailing_stop = min(trailing_stop, new_stop)
                if price > trailing_stop:
                    exit_time = row['DateTime']
                    pnl = position_size * (entry_price - price)  # 空仓盈亏计算
                    trades.append({
                        'entry_time': trade_entry_time,
                        'exit_time': exit_time,
                        'side': 'Short',
                        'entry_price': entry_price,
                        'exit_price': price,
                        'pnl': pnl
                    })
                    position = 0
                    trailing_stop = np.nan
    # 如果日末仍有持仓，则以当日最后一分钟价格平仓
    if position != 0:
        exit_time = day_df.iloc[-1]['DateTime']
        last_price = day_df.iloc[-1]['Close']
        if position == 1:
            pnl = position_size * (last_price - entry_price)
            trades.append({
                'entry_time': trade_entry_time,
                'exit_time': exit_time,
                'side': 'Long',
                'entry_price': entry_price,
                'exit_price': last_price,
                'pnl': pnl
            })
        else:
            pnl = position_size * (entry_price - last_price)
            trades.append({
                'entry_time': trade_entry_time,
                'exit_time': exit_time,
                'side': 'Short',
                'entry_price': entry_price,
                'exit_price': last_price,
                'pnl': pnl
            })
    return trades

# -------------------------------
# 主程序：加载数据、回测、汇总收益率
# -------------------------------
if __name__ == '__main__':
    # 读取SPY数据，假定文件为 spy_all.csv，包含 DateTime, Open, High, Low, Close, Volume
    spy_df = pd.read_csv('spy_all.csv', parse_dates=['DateTime'])
    spy_df.sort_values('DateTime', inplace=True)
    spy_df['Date'] = spy_df['DateTime'].dt.date
    spy_df['Time'] = spy_df['DateTime'].dt.strftime('%H:%M')
    spy_df['Open_day'] = spy_df.groupby('Date')['Open'].transform('first')
    spy_df['ret'] = spy_df['Close'] / spy_df['Open_day'] - 1

    # 利用透视表计算每个时点过去14天的平均绝对涨跌幅（σ）
    pivot = spy_df.pivot(index='Date', columns='Time', values='ret').abs()
    sigma = pivot.rolling(window=14, min_periods=1).mean()
    sigma = sigma.stack().reset_index(name='sigma')
    
    # 合并σ值
    spy_df = pd.merge(spy_df, sigma, on=['Date', 'Time'], how='left')
    
    # 计算每天的前一交易日收盘价，假定每天最后一笔价格为该日收盘价
    daily_close = spy_df.groupby('Date')['Close'].last()
    daily_close = daily_close.to_frame(name='close').reset_index()
    daily_close['prev_close'] = daily_close['close'].shift(1)
    # 将前一日收盘价合并到spy_df，按日期匹配
    spy_df = pd.merge(spy_df, daily_close[['Date', 'prev_close']], on='Date', how='left')
    # 对于首日或无前日数据时，取当日开盘价代替
    spy_df['prev_close'].fillna(spy_df['Open_day'], inplace=True)
    
    # 计算参考值：上限参考为 max(Open_day, prev_close)，下限参考为 min(Open_day, prev_close)
    spy_df['ref_upper'] = spy_df[['Open_day', 'prev_close']].max(axis=1)
    spy_df['ref_lower'] = spy_df[['Open_day', 'prev_close']].min(axis=1)
    
    # 依据论文公式计算Noise Area边界
    spy_df['UpperBound'] = spy_df['ref_upper'] * (1 + spy_df['sigma'])
    spy_df['LowerBound'] = spy_df['ref_lower'] * (1 - spy_df['sigma'])
    
    # 计算VWAP（依赖Volume字段）
    spy_df = spy_df.groupby('Date', group_keys=False).apply(compute_vwap)
    
    # 读取VIX数据，假定文件为 vix_all.csv，包含 DateTime, Open, High, Low, Close, Volume
    vix_df = pd.read_csv('vix_all.csv', parse_dates=['DateTime'])
    vix_df.sort_values('DateTime', inplace=True)
    vix_df['Date'] = vix_df['DateTime'].dt.date
    vix_df['Time'] = vix_df['DateTime'].dt.strftime('%H:%M')
    
    # 确定VIX数据最早可用日期
    vix_min_date = vix_df['Date'].min()
    print("VIX数据最早可用日期：", vix_min_date)
    
    # 定义允许交易的时间点（例如每半小时一次）
    allowed_times = ['09:30', '10:00', '10:30', '11:00', '11:30', 
                     '12:00', '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30']
    
    # 初始资金
    capital = 100000.0
    daily_results = []  # 存储每日回测结果
    all_trades = []     # 存储所有交易记录
    
    # 仅选择 spy 数据中日期 >= VIX 数据的最早日期
    unique_dates = sorted(spy_df['Date'].unique())
    unique_dates = [d for d in unique_dates if d >= vix_min_date]
    
    for trade_date in unique_dates:
        day_spy = spy_df[spy_df['Date'] == trade_date].copy().reset_index(drop=True)
        if day_spy.empty:
            continue
        
        # 获取该日VIX数据，优先取09:30之前的数据；若无则取当天第一条记录
        day_vix = vix_df[vix_df['Date'] == trade_date]
        pre_open_vix = day_vix[day_vix['Time'] < '09:30']
        if pre_open_vix.empty:
            if not day_vix.empty:
                pre_open_vix = day_vix.iloc[[0]]
            else:
                continue  # 如果当天没有VIX数据，直接跳过
        vix_value = pre_open_vix.iloc[-1]['Close']
        # 假设 VIX 数值以百分比表示，转换为日化波动率
        daily_vix = (vix_value / 100) / sqrt(252)
        target_vol = 0.02
        multiplier = min(4, target_vol / daily_vix)
        
        open_price = day_spy.iloc[0]['Open_day']
        position_size = floor(capital * multiplier / open_price)
        if position_size <= 0:
            continue
        
        trades = simulate_day(day_spy, allowed_times, position_size)
        day_pnl = sum(trade['pnl'] for trade in trades)
        capital_start = capital
        capital = capital + day_pnl
        daily_return = (capital - capital_start) / capital_start
        daily_results.append({'Date': trade_date, 'capital': capital, 'daily_return': daily_return})
        for t in trades:
            t['date'] = trade_date
            all_trades.append(t)
    
    # 构造每日结果 DataFrame
    daily_df = pd.DataFrame(daily_results)
    daily_df['Date'] = pd.to_datetime(daily_df['Date'])
    daily_df = daily_df.sort_values('Date').set_index('Date')
    
    # 按月汇总：以每个月初的资金为基准，计算月末资金变化率
    monthly = daily_df.resample('M').agg({'capital': ['first', 'last']})
    monthly.columns = ['month_start', 'month_end']
    monthly['monthly_return'] = monthly['month_end'] / monthly['month_start'] - 1
    
    # 按年汇总：同样计算每年的累计收益率
    yearly = daily_df.resample('A').agg({'capital': ['first', 'last']})
    yearly.columns = ['year_start', 'year_end']
    yearly['yearly_return'] = yearly['year_end'] / yearly['year_start'] - 1
    
    # 每个月只打印一行结果
    print("\n每个月累计收益率:")
    print(monthly[['month_start', 'month_end', 'monthly_return']])
    
    print("\n每年累计收益率:")
    print(yearly[['year_start', 'year_end', 'yearly_return']])
    
    total_return = capital / 100000.0 - 1
    print(f"\n整个回测期间累计收益率: {total_return*100:.2f}%")
